## Import Modules

Make sure that all modules are installed before.

In [1]:
from ecmwfapi import ECMWFDataServer
import ecmwfapi
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import re
import unittest
from unittest.mock import patch, MagicMock

In [2]:
server = ECMWFDataServer()

## Define functions

In [8]:
def download_ecmwf(date):
    '''Download the enfo total precipitation data from ECMWF via ECMWFAPI client for a specific date and the next 144 forecast timesteps.
    The input value date should be a string in the format 'YYYY-MM-DD'.'''
    
    # check if date has the correct format
    if not re.match(r'\d{4}-\d{2}-\d{2}', date):
        raise ValueError("Date has to be in the format: 'YYYY-MM-DD'.")
        
    year, month, day = date.split('-')
    filename = f'enfo_pf_{year}_{month}_{day}.nc'

    try:
        server.retrieve({
            "class": "s2",                # Dataset class
            "dataset": "s2s",             # Dataset name
            "date": date,                 # Date range
            "expver": "prod",             # Experiment version
            "levtype": "sfc",             # Level type
            "model": "glob",              # Model type
            "number": "1/to/100",                # Ensemble member numbers from 1 to 100
            "origin": "ecmf",             # Originating centre
            "param": "228228",            # Parameter (total precipitation)
            "step": "0/6/12/18/24/30/36/42/48/54/60/66/72/78/84/90/96/102/108/114/120/126/132/138/144", # Forecast steps in 6-hour intervals (maximum 1104)
            "stream": "enfo",             # Stream
            "time": "00:00:00",           # Forecast time (start)
            "type": "pf",                 # Forecast type (perturbed forecast)
            "format": "netcdf",           # Output format as netcdf
            "target": filename            # Target file name
        })
        print(f"Successfully downloaded data for {date} to {filename}")
    except ecmwfapi.api.APIException as e:
        raise RuntimeError(f"Download failed for {date}. APIException: {e}")
    except Exception as e:
        raise RuntimeError(f"Download failed for {date}. Error: {e}")

In [9]:
def download_ecmwf_cf(date):
    '''download enfo total precipitation data from ecmwf the control forecast for a specific date and the next 144 forecast steps.
    The input value date should be a string in the format 'YYYY-MM-DD'. '''
    if not re.match(r'\d{4}-\d{2}-\d{2}', date):
        raise ValueError("Date has to be in the format: 'YYYY-MM-DD'.")
        
    year, month, day = date.split('-')
    filename = f'enfo_cf_{year}_{month}_{day}.nc'

    try:
        server.retrieve({
            "class": "s2",
            "dataset": "s2s",
            "date": date,
            "expver": "prod",
            "levtype": "sfc",
            "model": "glob",
            "origin": "ecmf",
            "param": "228228",
            "step": "0/6/12/18/24/30/36/42/48/54/60/66/72/78/84/90/96/102/108/114/120/126/132/138/144",
            "stream": "enfo",
            "time": "00:00:00",
            "type": "cf",
            "target": filename
        })
        print(f"Successfully downloaded control forecast data from {date} to {filename}")
    
    except ecmwfapi.api.APIException as e:
        raise RuntimeError(f"Download failed for {date}. APIException: {e}")
    except Exception as e:
        raise RuntimeError(f"Download failed for {date}. Error: {e}")


In [10]:
from datetime import datetime, timedelta

def get_datelist(startdate, enddate):
    '''Returns a list of dates between startdate and enddate, inclusive.
    
    Input:
        startdate (str): Start date in the format 'YYYY-MM-DD'.
        enddate (str): End date in the format 'YYYY-MM-DD'.
    
    Returns:
        list: List of dates in the format 'YYYY-MM-DD'.
    
    Raises:
        ValueError: If startdate or enddate is not a string, or if dates are not in the correct format, 
                    or if enddate is before startdate.
    '''
    
    # Validation of the input parameters
    if not isinstance(startdate, str) or not isinstance(enddate, str):
        raise ValueError("Both startdate and enddate must be strings.")
    
    try:
        start_date = datetime.strptime(startdate, '%Y-%m-%d')
        end_date = datetime.strptime(enddate, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Invalid date format. Please use 'YYYY-MM-DD'.")
    
    if end_date < start_date:
        raise ValueError("End date cannot be before start date.")
    
    # Empty list, which will be returned at the end of function
    date_list = []
    
    # Loop from start_date to end_date
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)

    return date_list

## Unit Tests

In [11]:
# unit tests for function download_ecmwf
class TestDownloadECMWF(unittest.TestCase):
    @patch('ecmwfapi.api.ECMWFDataServer.retrieve')
    def test_download_success(self, mock_retrieve):
        # Simulate successful download
        mock_retrieve.return_value = None
        try:
            download_ecmwf('2024-05-13')
        except Exception as e:
            self.fail(f"download_ecmwf raised an exception unexpectedly: {e}")

    def test_invalid_date_format(self):
        with self.assertRaises(ValueError) as context:
            download_ecmwf('20240513')
        # Output of the actual error message for checking
        print(f"Actual error message: {context.exception}")
        self.assertTrue("Date has to be in the format: 'YYYY-MM-DD'." in str(context.exception))

    @patch('ecmwfapi.api.ECMWFDataServer.retrieve')
    def test_download_failure(self, mock_retrieve):
        # Simulate API errors
        mock_retrieve.side_effect = ecmwfapi.api.APIException("Some API error")
        with self.assertRaises(RuntimeError) as context:
            download_ecmwf('2024-05-13')
        # Output of the actual error message for checking
        print(f"Actual error message: {context.exception}")
        self.assertTrue("Download failed for 2024-05-13. APIException: 'Some API error'" in str(context.exception))

    @patch('ecmwfapi.api.ECMWFDataServer.retrieve')
    def test_general_exception(self, mock_retrieve):
        # Simulate general error
        mock_retrieve.side_effect = Exception("Some general error")
        with self.assertRaises(RuntimeError) as context:
            download_ecmwf('2024-05-13')
        # Output of the actual error message for checking
        print(f"Actual error message: {context.exception}")
        self.assertTrue("Download failed for 2024-05-13. Error: Some general error" in str(context.exception))


In [12]:
# unit tests for download_ecmwf_cf
class TestDownloadECMWFCF(unittest.TestCase):

    @patch.object(ECMWFDataServer, 'retrieve')
    def test_download_success(self, mock_retrieve):
        # Simulate successful download
        date = '2024-05-13'
        expected_filename = f'enfo_cf_2024_05_13.nc'

        mock_retrieve.return_value = None

        try:
            download_ecmwf_cf(date)
        except Exception as e:
            self.fail(f"Exception raised: {e}")

        # Check whether the retrieve method was called with the correct parameters
        mock_retrieve.assert_called_once()
        call_args = mock_retrieve.call_args[0][0]
        self.assertEqual(call_args['date'], date)
        self.assertEqual(call_args['type'], 'cf')
        self.assertEqual(call_args['target'], expected_filename)

    def test_invalid_date_format(self):
        # Test of invalid date format
        invalid_date = '20240513'
        with self.assertRaises(ValueError) as context:
            download_ecmwf_cf(invalid_date)
        self.assertEqual(str(context.exception), "Date has to be in the format: 'YYYY-MM-DD'.")

    @patch('ecmwfapi.api.ECMWFDataServer.retrieve')
    def test_download_failure(self, mock_retrieve):
        # Simulate download error
        mock_retrieve.side_effect = ecmwfapi.api.APIException("Some API error")
        date = '2024-05-13'
        
        with self.assertRaises(RuntimeError) as context:
            download_ecmwf_cf(date)
        # Output of the actual error message for checking
        print(f"Actual error message: {context.exception}")
        self.assertIn(f"Download failed for {date}. APIException: 'Some API error'", str(context.exception))

    @patch('ecmwfapi.api.ECMWFDataServer.retrieve')
    def test_general_error(self, mock_retrieve):
        # Simulate general error
        mock_retrieve.side_effect = ValueError("Some general error")
        date = '2024-05-13'
        
        with self.assertRaises(RuntimeError) as context:
            download_ecmwf_cf(date)
        # Output of the actual error message for checking
        print(f"Actual error message: {context.exception}")
        self.assertIn(f"Download failed for {date}. Error: Some general error", str(context.exception))

        self.assertIn(f"Download failed for {date}. Error: Some general error", str(context.exception))

In [13]:
# unit tests for function get_datelist
class TestGetDateList(unittest.TestCase):
    
    def test_valid_dates(self):
        result = get_datelist('2024-05-13', '2024-05-19')
        expected = [
            '2024-05-13',
            '2024-05-14',
            '2024-05-15',
            '2024-05-16',
            '2024-05-17',
            '2024-05-18',
            '2024-05-19'
        ]
        self.assertEqual(result, expected)
    
    def test_same_start_end_date(self):
        result = get_datelist('2024-05-13', '2024-05-13')
        expected = ['2024-05-13']
        self.assertEqual(result, expected)
    
    def test_end_before_start_date(self):
        with self.assertRaises(ValueError) as context:
            get_datelist('2024-05-19', '2024-05-13')
        self.assertEqual(str(context.exception), "End date cannot be before start date.")
    
    def test_invalid_date_format(self):
        with self.assertRaises(ValueError) as context:
            get_datelist('20240513', '2024-05-19')
        self.assertEqual(str(context.exception), "Invalid date format. Please use 'YYYY-MM-DD'.")
    
    def test_invalid_start_date_format(self):
        with self.assertRaises(ValueError) as context:
            get_datelist('2024-05-100', '2024-05-13')
        self.assertEqual(str(context.exception), "Invalid date format. Please use 'YYYY-MM-DD'.")
    
    def test_invalid_end_date_format(self):
        with self.assertRaises(ValueError) as context:
            get_datelist('2024-05-13', '2024-0519')
        self.assertEqual(str(context.exception), "Invalid date format. Please use 'YYYY-MM-DD'.")
    
    def test_non_string_input(self):
        with self.assertRaises(ValueError) as context:
            get_datelist(20240513, '2024-05-19')
        self.assertEqual(str(context.exception), "Both startdate and enddate must be strings.")

In [14]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...............
----------------------------------------------------------------------
Ran 15 tests in 0.020s

OK


Actual error message: Download failed for 2024-05-13. APIException: 'Some API error'
Successfully downloaded data for 2024-05-13 to enfo_pf_2024_05_13.nc
Actual error message: Download failed for 2024-05-13. Error: Some general error
Actual error message: Date has to be in the format: 'YYYY-MM-DD'.
Actual error message: Download failed for 2024-05-13. APIException: 'Some API error'
Successfully downloaded control forecast data from 2024-05-13 to enfo_cf_2024_05_13.nc
Actual error message: Download failed for 2024-05-13. Error: Some general error


## Download data  
The heavy rain event in southern Germany was at the weekend of the 18th may 2024. We are looking at the forecast data beginning from the 13th may 2024. 

In [15]:
datelist = get_datelist('2024-05-13','2024-05-18')

In [16]:
# download the ensemble data and the control forecast
for date in datelist:
    download_ecmwf(date)
    download_ecmwf_cf(date)

2024-06-27 13:52:23 ECMWF API python library 1.6.3
2024-06-27 13:52:23 ECMWF API at https://api.ecmwf.int/v1
2024-06-27 13:52:23 Welcome Luisa Reske
2024-06-27 13:52:24 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web+API+FAQ or contact servicedesk@ecmwf.int
2024-06-27 13:52:24 ------------ WARNING ------------
2024-06-27 13:52:24 Access to this dataset is transitioning to a new interface, dates to be announced soon
2024-06-27 13:52:24 For more information on how to access this data in the future, visit https://confluence.ecmwf.int/x/-wUiEw
2024-06-27 13:52:24 ---------------------------------
2024-06-27 13:52:24 Request submitted
2024-06-27 13:52:24 Request id: 667d6e982697a9d5b8a89a8b
2024-06-27 13:52:24 Request is submitted
2024-06-27 13:52:26 Calling 'nice mars /tmp/20240627-1350/99/tmp-_mars-G8il_0.req'
2024-06-27 13:52:26 Forcing MIR_CACHE_PATH=/data/ec_coeff
2024-06-27 13:52:26 mars - WARN -
2024-06-27 13:52:26 mars - WARN -
2024-06-27 13:52:26 M